In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
! pip install spacy_transformers
! pip install -U spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53 kB 2.6 MB/s 
     |████████████████████████████████| 1.1 MB 62.0 MB/s 
     |████████████████████████████████| 4.4 MB 58.6 MB/s 
     |████████████████████████████████| 101 kB 13.1 MB/s 
     |████████████████████████████████| 596 kB 46.7 MB/s 
     |████████████████████████████████| 6.6 MB 55.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

In [4]:
!git clone https://github.com/higher4comp/project.git

Cloning into 'project'...
remote: Enumerating objects: 98, done.
remote: Total 98 (delta 0), reused 0 (delta 0), pack-reused 98
Unpacking objects: 100% (98/98), done.


In [5]:
%cd /content/project/
!git pull

/content/project
Already up to date.


In [6]:
import json
cv_data = json.load(open('/content/project/annotation/final_resumes.json','r'))

In [7]:
!python -m spacy init fill-config /content/project/base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [8]:
def get_spacy_doc(file, data):
  nlp=spacy.blank('en')
  db=DocBin()

  for item in data:
    text = item['text']
    doc=nlp.make_doc(text)
    annot=item['label']

    
    ents=[]
    entity_indices=[]

    for start, end, label in annot:
      skip_entity=False
      for idx in range(start, end):
        if idx in entity_indices:
          skip_entity=True
          break
      if skip_entity==True:
         continue
        
      entity_indices = entity_indices + list(range(start, end))

      try:
        span=doc.char_span(start,end,label=label,alignment_mode='contract')
      except:
        continue
        
      if span is None:
        err_data=str([start,end]) + "     " + str(text) + "\n"
        file.write(err_data)
        
      else:
        ents.append(span)

    try:
        doc.ents=ents
        db.add(doc)
    except:
      pass


  return db

In [9]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(cv_data, test_size=0.3)

In [10]:
file = open('error.txt', 'w')
db = get_spacy_doc(file, train)
db.to_disk('train_data.spacy')

db = get_spacy_doc(file, test)
db.to_disk('test_data.spacy') 

file.close()

In [11]:
! python -m spacy train /content/project/config.cfg --output ./output --paths.train ./train_data.spacy --paths.dev ./test_data.spacy --gpu-id 0

✔ Created output directory: output
ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2022-08-09 18:05:49,346] [INFO] Set up nlp object from config
[2022-08-09 18:05:49,356] [INFO] Pipeline: ['transformer', 'ner']
[2022-08-09 18:05:49,360] [INFO] Created vocabulary
[2022-08-09 18:05:49,361] [INFO] Finished initializing nlp object
Downloading: 100% 481/481 [00:00<00:00, 467kB/s]
Downloading: 100% 878k/878k [00:01<00:00, 674kB/s] 
Downloading: 100% 446k/446k [00:01<00:00, 411kB/s]
Downloading: 100% 1.29M/1.29M [00:01<00:00, 1.01MB/s]
Downloading: 100% 478M/478M [00:06<00:00, 74.3MB/s]
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a mode

In [12]:
nlp = spacy.load('/content/project/output/model-best')

In [13]:
!pip install PyMuPDF

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 8.8 MB 19.3 MB/s 


In [14]:
import sys, fitz

In [15]:
for x in range(1,11):
  fname='/content/project/resume_test/'+str(x)+'.pdf'
  doc=fitz.open(fname)
  print('resume num '+str(x)+' test')
  text=" "
  for page in doc:
    text=text+str(page.get_text())
  text=text.strip()
  text=' '.join(text.split())
  doc=nlp(text)
  for ent in doc.ents:
    print(ent.text, " ->>>>>> ", ent.label_)
  print('\n')

resume num 1 test
Java Developer Nsoft technologies - Pune, Maharashtra May 2016 to Present  ->>>>>>  work experience
BE in  ->>>>>>  major
Information Technology University  ->>>>>>  major
c++  ->>>>>>  skill
Core Java  ->>>>>>  skill
hibernate  ->>>>>>  skill
spring  ->>>>>>  skill
Jsp  ->>>>>>  skill
Servlet  ->>>>>>  skill
mysql  ->>>>>>  skill
Web Services  ->>>>>>  skill
spring mvc  ->>>>>>  skill
Ejb  ->>>>>>  skill
jms Certifications/Licenses International Journal of Research in information Technology  ->>>>>>  skill
C++  ->>>>>>  skill
Java  ->>>>>>  skill
Html  ->>>>>>  skill
Jsp  ->>>>>>  skill
Servlet  ->>>>>>  skill
Spring mvc  ->>>>>>  skill
Hibernet  ->>>>>>  skill
Spring Web  ->>>>>>  skill
Tomcat 7.0  ->>>>>>  skill
Windows 7  ->>>>>>  skill
Windows8  ->>>>>>  skill
Unix  ->>>>>>  skill
Eclipse  ->>>>>>  skill
NetBean  ->>>>>>  skill


resume num 2 test
Java developer  ->>>>>>  ability
Java j2ee senior developer Senior Java Developer Mepra software  ->>>>>>  ability
M.